# Genetic algo for lighthouse sensor distribution on arbitrary mesh

In [1]:
import numpy as np
from stl import mesh as meshstl
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from plotmesh import plot_mesh
import math
import random
from pyquaternion import Quaternion
from scipy.linalg import qr
import roslib
import rospy
import math
import tf

In [2]:
stl_file = 'roboy_models/TestCube/stls/testcube_40mm.stl'

In [3]:
#Move Lighthouses to
translationLH1 = [-2.,0,2.]
quat1 = Quaternion(axis=[0,0,1],angle=-np.pi / 2)

global LH1 
LH1 = (translationLH1, quat1)

translationLH2 = [2,0.,2.]
quat2 = Quaternion(axis=[0,0,1], angle=np.pi / 2)

global LH2
LH2 = (translationLH2, quat2)

print(LH1); print(LH2)

([-2.0, 0, 2.0], Quaternion(0.70710678118654757, -0.0, -0.0, -0.70710678118654746))
([2, 0.0, 2.0], Quaternion(0.70710678118654757, 0.0, 0.0, 0.70710678118654746))


In [4]:
from visualization_msgs.msg import Marker, MarkerArray
from data.trafomatrix import get_random_trafomatrix

scale = 0.01
duration = 1
pos = [0,0,0]
orientationMesh = Quaternion(axis=(1,0,0),angle = np.pi*0)

rospy.init_node('fixed_tf_broadcaster')

#Visualization mesh
topic = 'visualization_marker'
publisher = rospy.Publisher(topic, Marker, queue_size=1)
rate2 = rospy.Rate(1)

In [5]:
while not rospy.is_shutdown():
    
    for i in range(3):
        mesh = Marker()
        mesh.header.frame_id = 'world';
        mesh.ns = 'ns';
        mesh.type = Marker.MESH_RESOURCE
        mesh.color.r = 1.0
        mesh.color.g = 1.0
        mesh.color.b = 1.0
        mesh.color.a = 0.9
        mesh.scale.x = scale
        mesh.scale.y = scale
        mesh.scale.z = scale
        mesh.lifetime = rospy.Duration(-1);
        mesh.header.stamp = rospy.Time.now()
        mesh.action = Marker.ADD
        mesh.id = 10000000;
        mesh.pose.position.x = pos[0];
        mesh.pose.position.y = pos[1];
        mesh.pose.position.z = pos[2];
        mesh.pose.orientation.x = orientationMesh[1];
        mesh.pose.orientation.y = orientationMesh[2];
        mesh.pose.orientation.z = orientationMesh[3];
        mesh.pose.orientation.w = orientationMesh[0];
        mesh.mesh_resource = ("package://" + stl_file)
        #mesh.mesh_resource = "package://roboy_models/roboy_2_0_simplified/meshes/CAD/head.stl"

        publisher.publish(mesh);
        rate2.sleep()
        #MeshMarker.type = visualization_msgs::Marker::MESH_RESOURCE;
        #MeshMarker.mesh_resource = "package://pr2_description/meshes/base_v0/base.dae";
    break


# Preprocess data

In [6]:
#Get mesh vertices and normals
mesh1 = meshstl.Mesh.from_file(('../'+ stl_file))
#mesh1 = meshstl.Mesh.from_file('../src/roboy_models/roboy_2_0_simplified/meshes/CAD/torso.stl')

global triangles 
triangles = scale * np.matrix(mesh1.points)


vertices = np.reshape(triangles,(len(triangles)*3,3)) 

#normals = np.zeros_like((len(triangles),3))
normalsNotNorm = mesh1.normals

normals = []
for normal in normalsNotNorm:
    normals.append(1/np.linalg.norm(normal,2)*normal)
    
normals = np.matrix(normals)
normals = scale * normals

#Initialize sensors in centers of triangle
sensors = (triangles[:,0:3]+triangles[:,3:6]+triangles[:,6:9])/3

print('%d triangles' %len(triangles))
#print(triangles)
print('')
print('%d vertices' %len(vertices))
#print(vertices)
print('')
print('%d normals normiert' %len(normals))
#print(normals)
print('')
print('%d sensors in centers of triangles' %len(sensors))
#print(sensors)
#print(vertices); print(normals); print(sensors)

12 triangles

36 vertices

12 normals normiert

12 sensors in centers of triangles


In [7]:
#Prepare vertices, normals and sensor locations for Transformation

verticesRot = np.insert(vertices, 3,values=1,axis=1)
#print(verticesRot)
#print('')

normalsRot = np.insert(normals, 3, values=0, axis=1)
#print(normalsRot)
#print('')

sensorsRot = np.insert(vertices, 3,values=1,axis=1)
#print(sensorsRot)
#print('')

In [8]:
#Test transform vertices
T = get_random_trafomatrix()
newvertices = np.dot(T,np.transpose(verticesRot)).T


#print(newvertices)

In [9]:
#Sensor visualization in RVIZ


for i,sensor in enumerate(sensors):

    sensor = np.squeeze(np.array(sensor))
    
    rate3 = rospy.Rate(100)
    while not rospy.is_shutdown():

        markers = Marker()
        markers.header.frame_id = 'world'
        markers.ns = 'ns'
        markers.type = Marker.SPHERE

        markers.pose.position.x = sensor[0];
        markers.pose.position.y = sensor[1];
        markers.pose.position.z = sensor[2];
        markers.pose.orientation.x = 0
        markers.pose.orientation.y = 0
        markers.pose.orientation.z = 0
        markers.pose.orientation.w = 0

        #print(marker.points)
        markers.color.r = 0.0
        markers.color.g = 0.0
        markers.color.b = 1.0
        markers.color.a = 1.0

        markers.scale.x = 0.01
        markers.scale.y = 0.01
        markers.scale.z = 0.01
        markers.lifetime = rospy.Duration(-1);
        markers.header.stamp = rospy.Time.now()
        markers.action = Marker.ADD
        markers.id = 100+i

        #mesh.mesh_resource = "package://common_utilities/stls/testcube_40mm.stl"
        #mesh.mesh_resource = "package://roboy_models/roboy_2_0_simplified/meshes/CAD/head.stl"

        publisher.publish(markers);
        rate3.sleep()
        break


# GA

In [10]:
from deap import algorithms, base, creator, tools

In [11]:
#sensors to dict
sensorList = []

for sensor in sensors:
    sensor = np.squeeze(np.array(sensor)).tolist()
    sensorList.append(sensor)

sensor_dict =  list(zip(range(len(sensors)), sensorList))

In [12]:
creator.create("FitnessMax", base.Fitness, weights=(0.001,1.0,1.0)) # 1 -> maximum probblem
creator.create("Individual", list, fitness=creator.FitnessMax)

In [13]:
toolbox = base.Toolbox()

In [14]:
# Find random sensors
def randomSensor(v):
    i = random.randint(0,len(v)-1)
    #print(i)
    rand = v[i]
    
    return rand[0]

In [15]:
r = np.squeeze(np.array(randomSensor(sensors)))

print(r)

[-0.06666667 -0.06666667 -0.2       ]


In [16]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", randomSensor, sensor_dict)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 4)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [17]:
toolbox.attr_bool()

0

In [18]:
toolbox.individual()

[5, 1, 3, 9]

In [19]:
def triMinDistanceToLighthouse(origin, sensor, tri, LH_CSframe):
       
    ray = sensor - origin
    v0 = tri[0:3]
    v1 = tri[3:6]
    v2 = tri[6:9]
    
    e1 = v1 - v0
    e2 = v2 - v0
    #print('test')
    #print(v0); print(v1); print(v2); print(e1); print(e2) 
    
    h = np.cross(ray, e2)
    
    a = e1.dot(h)
    if(a > -0.00001 and a < 0.00001):
        return False
    
    f = 1/a
    s = origin - v0
    u = f * (s.dot(h))
    
    if (u < 0.0 or u > 1.0):
        return False
    
    q = np.cross(s, e1)
    v = f * (ray.dot(q))
    
    if (v < 0.0 or u+v > 1.0):
        return False
    
    t = f * (e2.dot(q))
    if (t > 0.00001):            
        return t
    else:
        return False



def SortTriangles():
    triangles = []

# Evaluation (Fitness) Function

In [20]:
from data.RayIntersectsTriangle import rayIntersectsTriangle,rayIntersectsTriangleVisualization

def FitnessFunction(sensors):

    br = tf.TransformBroadcaster()
    br.sendTransform((LH1[0][0], LH1[0][1], LH1[0][2]),
                     (quat1[0], quat1[1], quat1[2], quat1[3]),
                     rospy.Time.now(),
                     "lighthouse1",
                     "world")
    br.sendTransform((LH2[0][0], LH2[0][1], LH2[0][2]),
                     (quat2[0], quat2[1], quat2[2], quat2[3]),
                     rospy.Time.now(),
                     "lighthouse2",
                     "world")

    
    #1. COMPUTE VISIBLE SENSORS AT THE MOMENT
    LH1_array = np.asarray(LH1[0])
    LH2_array = np.asarray(LH2[0])
    #testTriangle = np.squeeze(np.asarray(triangles[0]))

    visibleLH1 = 0
    visibleLH2 = 0

    for i, nmbr_sensor in enumerate(sensors):
        sensor = sensor_dict[nmbr_sensor][1]

        #get distance of current sensor and check if intersection
        interDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                             np.squeeze(np.asarray(triangles[nmbr_sensor])), 'lighthouse1')
        interDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                             np.squeeze(np.asarray(triangles[nmbr_sensor])), 'lighthouse2')
        
        #print('interDist');print(interDistLH1);print(interDistLH2);print('endinterDist')

        isVisible1 = True
        isVisible2 = True
        
        for j in range(len(triangles)):
            if(nmbr_sensor != j):
                #print(j)
                if(isVisible1):
                    newInterDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                                        np.squeeze(np.asarray(triangles[j])), 'lighthouse1')
                    if(newInterDistLH1 < interDistLH1 and newInterDistLH1 != False):
                        isVisible1 = False
                        #interDistLH1 = newInterDistLH1
                        #visibleLH1 += 1
                if(isVisible2):
                    newInterDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                                        np.squeeze(np.asarray(triangles[j])), 'lighthouse2')
                    if(newInterDistLH2 < interDistLH2 and newInterDistLH2 != False):
                        isVisible2 = False
                        #interDistLH2 = newInterDistLH2
                        #visibleLH2 += 1
                    
                if(not (isVisible1 or isVisible2)):
                    break
        
        if(isVisible1):
            visibleLH1 += 1
        if(isVisible2):
            visibleLH2 += 1
        #print(newInterDistLH1); print(newInterDistLH2)
        
    #2. COMPUTE EUCLIDEAN DISTANCE OF SENSORS
    individual = sensors
    dist = 0
    for i,ind in enumerate(individual):
        ind = np.asarray(sensor_dict[ind][1])
        for j in range(i,len(individual)):
            if(i != j):
                indivi = np.asarray(sensor_dict[individual[j]][1])
                dist += np.linalg.norm(ind-indivi)
    #print('visiblesensors')
    #print(visibleLH1);print(visibleLH2);print('')
    
    return dist, visibleLH1, visibleLH2
    #COST FUNCTION?
    #return dist, np.sqrt(visibleLH1**2 + visibleLH2**2)

In [21]:
toolbox.register("evaluate", FitnessFunction)

toolbox.register("mate", tools.cxTwoPoint)
# Independent probability  : for each attribute to be mutated.# low~up rondom int
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(sensorList)-1, indpb=0.2) 
toolbox.register("select", tools.selTournament, tournsize=3)

In [22]:
# Creating population

population = toolbox.population(n=10)

In [23]:
hof = tools.HallOfFame(1)

In [24]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [25]:
population, log = algorithms.eaSimple(population, 
                                      toolbox, 
                                      cxpb=0.5, 
                                      mutpb=0.2, 
                                      ngen=1000, 
                                      stats=stats, 
                                      halloffame=hof, 
                                      verbose=True)

gen	nevals	avg    	std    	min	max
0  	10    	1.31415	0.81919	0  	3  
1  	5     	1.31389	0.634784	0  	2.06048
2  	6     	1.30358	0.512   	0  	2.06048
3  	6     	1.31614	0.637098	0  	2.06048
4  	6     	1.24337	0.660356	0  	2.06048
5  	8     	1.23543	0.718745	0  	2.06048
6  	8     	1.34599	0.554264	0  	2.10577
7  	3     	1.38513	0.507589	1  	2.10577
8  	2     	1.36557	0.517102	1  	2.10577
9  	9     	1.45018	0.517648	1  	2.10577
10 	3     	1.3618 	0.512879	1  	2.10577
11 	6     	1.28853	0.60347 	0  	2.10577
12 	7     	1.35965	0.510736	1  	2.10577
13 	6     	1.36859	0.521267	1  	2.10577
14 	7     	1.32684	0.563556	0  	2.10577
15 	6     	1.3971 	0.522731	1  	2.10577
16 	4     	1.36859	0.521267	1  	2.10577
17 	6     	1.36859	0.521267	1  	2.10577
18 	7     	1.32387	0.558598	0  	2.10577
19 	6     	1.39806	0.523796	1  	2.10577
20 	1     	1.36859	0.521267	1  	2.10577
21 	7     	1.33358	0.570977	0  	2.10577
22 	8     	1.36859	0.521267	1  	2.10577
23 	3     	1.36859	0.521267	1  	2.10577
24 	10    

206	8     	1.35447	0.508702	1  	2.11334
207	4     	1.36746	0.520019	1  	2.11334
208	8     	1.35827	0.511044	1  	2.11334
209	8     	1.37111	0.524831	1  	2.11334
210	5     	1.43413	0.532585	1  	2.11334
211	3     	1.37111	0.524831	1  	2.11334
212	7     	1.42646	0.524565	1  	2.11334
213	4     	1.34832	0.498971	1  	2.11334
214	8     	1.4016 	0.528334	1  	2.11334
215	8     	1.32677	0.564223	0  	2.11334
216	8     	1.43711	0.535999	1  	2.11334
217	7     	1.37111	0.524831	1  	2.11334
218	5     	1.33366	0.571417	0  	2.11334
219	5     	1.37111	0.524831	1  	2.11334
220	9     	1.39622	0.522601	1  	2.11334
221	8     	1.37111	0.524831	1  	2.11334
222	4     	1.37111	0.524831	1  	2.11334
223	8     	1.38245	0.508583	1  	2.11334
224	3     	1.32758	0.565337	0  	2.11334
225	4     	1.33642	0.561659	0  	2.11334
226	8     	1.37111	0.524831	1  	2.11334
227	4     	1.43542	0.53401 	1  	2.11334
228	6     	1.37111	0.524831	1  	2.11334
229	6     	1.40389	0.620021	0  	3      
230	6     	1.37111	0.524831	1  	2.11334


413	7     	1.29792	0.614676	0  	2.11334
414	4     	1.37111	0.524831	1  	2.11334
415	7     	1.41453	0.513279	1  	2.11334
416	8     	1.36918	0.522202	1  	2.11334
417	6     	1.37111	0.524831	1  	2.11334
418	2     	1.37111	0.524831	1  	2.11334
419	6     	1.40209	0.528914	1  	2.11334
420	5     	1.37111	0.524831	1  	2.11334
421	1     	1.33273	0.570371	0  	2.11334
422	6     	1.37111	0.524831	1  	2.11334
423	9     	1.35634	0.508295	1  	2.11334
424	6     	1.41352	0.513689	1  	2.11334
425	7     	1.37056	0.524053	1  	2.11334
426	8     	1.4008 	0.527394	1  	2.11334
427	3     	1.37111	0.524831	1  	2.11334
428	4     	1.31781	0.558886	0  	2.11334
429	5     	1.40209	0.528914	1  	2.11334
430	7     	1.37111	0.524831	1  	2.11334
431	5     	1.42984	0.528477	1  	2.11334
432	6     	1.28939	0.607755	0  	2.11334
433	3     	1.42945	0.528158	1  	2.11334
434	4     	1.42945	0.528158	1  	2.11334
435	10    	1.2084 	0.668073	0  	2.11334
436	6     	1.35238	0.502553	1  	2.11334
437	3     	1.37111	0.524831	1  	2.11334


620	4     	1.42517	0.522769	1  	2.11334
621	5     	1.39508	0.520284	1  	2.11334
622	10    	1.36632	0.579229	0  	2.11334
623	3     	1.37111	0.524831	1  	2.11334
624	6     	1.37111	0.524831	1  	2.11334
625	5     	1.4281 	0.527097	1  	2.11334
626	4     	1.37111	0.524831	1  	2.11334
627	8     	1.37111	0.524831	1  	2.11334
628	9     	1.46827	0.59538 	1  	3      
629	4     	1.46827	0.59538 	1  	3      
630	8     	1.4008 	0.527394	1  	2.11334
631	1     	1.33246	0.570067	0  	2.11334
632	4     	1.37111	0.524831	1  	2.11334
633	3     	1.45963	0.528215	1  	2.11334
634	6     	1.46   	0.528243	1  	2.11334
635	6     	1.22103	0.683014	0  	2.11334
636	6     	1.33142	0.568967	0  	2.11334
637	6     	1.37111	0.524831	1  	2.11334
638	1     	1.43607	0.53476 	1  	2.11334
639	7     	1.37111	0.524831	1  	2.11334
640	4     	1.36827	0.521024	1  	2.11334
641	9     	1.36827	0.521024	1  	2.11334
642	5     	1.27199	0.601622	0  	2.11334
643	6     	1.37111	0.524831	1  	2.11334
644	4     	1.55051	0.579324	1  	3      


808	4     	1.26189	0.605085	0       	2.11334
809	6     	1.28939	0.607755	0       	2.11334
810	3     	1.40377	0.531028	1       	2.11334
811	7     	1.39161	0.516758	1       	2.11334
812	5     	1.35859	0.51128 	1       	2.11334
813	8     	1.33246	0.570067	0       	2.11334
814	4     	1.33457	0.572356	0       	2.11334
815	6     	1.37111	0.524831	1       	2.11334
816	3     	1.37111	0.524831	1       	2.11334
817	8     	1.36214	0.514258	1       	2.11334
818	5     	1.40372	0.530963	1       	2.11334
819	10    	1.37111	0.524831	1       	2.11334
820	6     	1.37111	0.524831	1       	2.11334
821	2     	1.37111	0.524831	1       	2.11334
822	5     	1.32488	0.617956	0       	2.11334
823	1     	1.29519	0.6121  	0       	2.11334
824	10    	1.40209	0.528914	1       	2.11334
825	6     	1.2176 	0.678562	0       	2.11334
826	7     	1.4519 	0.519663	1       	2.11334
827	5     	1.35706	0.50752 	1       	2.11334
828	6     	1.33246	0.570067	0       	2.11334
829	5     	1.35144	0.501665	1       	2.11334
830	7     

993	10    	1.70649	0.501956	1       	2.12164
994	6     	1.59964	0.592926	0       	2.12164
995	9     	1.73857	0.486024	1       	2.12164
996	5     	1.70721	0.502536	1       	2.12164
997	4     	1.75175	0.525902	1       	3      
998	9     	1.62451	0.574949	0       	2.12164
999	5     	1.69333	0.496613	1       	2.12164
1000	9     	1.72567	0.478276	1       	2.12164


In [26]:
best_sensors = tools.selBest(population, k=5)
print(best_sensors)

[[7, 9, 3, 11], [7, 9, 3, 11], [7, 9, 3, 11], [7, 9, 3, 11], [7, 9, 3, 11]]


In [27]:
#Sensor visualization in RVIZ

for i, best in enumerate(best_sensors[0]):

    show_sensor = sensor_dict[best][1]
    
    rate3 = rospy.Rate(10)
    while not rospy.is_shutdown():

        markers = Marker()
        markers.header.frame_id = 'world'
        markers.ns = 'ns'
        markers.type = Marker.SPHERE

        markers.pose.position.x = show_sensor[0];
        markers.pose.position.y = show_sensor[1];
        markers.pose.position.z = show_sensor[2];
        markers.pose.orientation.x = 0
        markers.pose.orientation.y = 0
        markers.pose.orientation.z = 0
        markers.pose.orientation.w = 0

        #print(marker.points)
        markers.color.r = 1.0
        markers.color.g = 0.0
        markers.color.b = 0.0
        markers.color.a = 1.0

        markers.scale.x = 0.015
        markers.scale.y = 0.015
        markers.scale.z = 0.015
        markers.lifetime = rospy.Duration(-1);
        markers.header.stamp = rospy.Time.now()
        markers.action = Marker.ADD
        markers.id = 100+i


        publisher.publish(markers);
        rate3.sleep()
        break
